In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from arguments import Arguments
from svm import SVM
from distributor import get_distributed_data, get_distributed_data_using_loader
from distributor import get_fog_graph
import os
import pickle as pkl
import syft as sy
import torch
from torchvision import datasets, transforms
from train import fl_train as train, fog_train
from train import test

In [4]:
# Setups
args = Arguments()
USE_CUDA = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
device = torch.device("cuda" if USE_CUDA else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if USE_CUDA else {}
kwargs = {}

In [5]:
ckpt_path = '../ckpts'
dataset = 'mnist'
clf_type = 'svm'
paradigm = 'fl_stratified_num_workers_{}_lr_{}_batch_{}'.format(
    args.num_workers,
    args.lr,
    args.batch_size
)
model_name = '{}_{}_{}'.format(dataset, clf_type, paradigm)
print(model_name)
init_path = '../init/mnist_svm.init'
best_path = os.path.join(ckpt_path, model_name + '.best')
stop_path = os.path.join(ckpt_path, model_name + '.stop')


mnist_svm_fl_stratified_num_workers_50_lr_0.03_batch_1200


In [6]:
# Setup hook to support FL
hook = sy.TorchHook(torch)
# Define workers
workers = list()
for id_ in range(args.num_workers):
    workers.append(sy.VirtualWorker(hook, id=str(id_)))

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.num_test, shuffle=True, **kwargs)


In [8]:
# prepare graph and data
fog_graph, workers = get_fog_graph(hook, args.num_workers, args.num_clusters,
                                   args.shuffle_workers, args.uniform_clusters, fog=False)
X_trains, y_trains = pkl.load(open('../ckpts/data_non_iid_10_stratify_True_uniform_True.pkl', 'rb'))

print(fog_graph)

{'L1_W0': ['L0_W0', 'L0_W1', 'L0_W2', 'L0_W3', 'L0_W4', 'L0_W5', 'L0_W6', 'L0_W7', 'L0_W8', 'L0_W9', 'L0_W10', 'L0_W11', 'L0_W12', 'L0_W13', 'L0_W14', 'L0_W15', 'L0_W16', 'L0_W17', 'L0_W18', 'L0_W19', 'L0_W20', 'L0_W21', 'L0_W22', 'L0_W23', 'L0_W24', 'L0_W25', 'L0_W26', 'L0_W27', 'L0_W28', 'L0_W29', 'L0_W30', 'L0_W31', 'L0_W32', 'L0_W33', 'L0_W34', 'L0_W35', 'L0_W36', 'L0_W37', 'L0_W38', 'L0_W39', 'L0_W40', 'L0_W41', 'L0_W42', 'L0_W43', 'L0_W44', 'L0_W45', 'L0_W46', 'L0_W47', 'L0_W48', 'L0_W49']}


In [9]:
best = 0
# Fire the engines
model = SVM().to(device)

model.load_state_dict(torch.load(init_path))
print('Load init: {}'.format(init_path))


Load init: ../init/mnist_svm.init


In [ ]:
best = 0

x_ax = []
y_ax = []
for epoch in range(1, args.epochs + 1):
    train(args, model, fog_graph, workers, X_trains, y_trains,
          device, epoch, loss_fn='hinge')
    acc = test(args, model, device, test_loader, best, epoch, loss_fn='hinge')
    x_ax.append(epoch)
    y_ax.append(acc)

    if args.save_model and acc > best:
        best = acc
        torch.save(model.state_dict(), best_path)
        print('Model best  @ {}, acc {}: {}\n'.format(epoch, acc, best_path))
    
if (args.save_model):
    torch.save(model.state_dict(), stop_path)
    print('Model stop: {}'.format(stop_path))


In [ ]:
import pickle as pkl
pkl.dump((x_ax, y_ax), open('../ckpts/history_{}.pkl'.format(model_name), 'wb'))  


In [ ]:
import matplotlib.pyplot as plt
plt.plot(x_ax, y_ax)
plt.savefig('../plots/{}.png'.format(model_name))


In [ ]:
model_name